In [1]:
### A3. Membuat dataset profile debitur selama 1 tahun ###
### -------------------------------------------------- ###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from pandas.plotting import radviz

In [4]:
# Source directory and result #
src = r'SampleDataCsv.csv'
df = pd.read_csv(src, sep='|')

str1 = 'AsofDate'
str2 = 'CIF'
str3 = 'Collectibility'
str4 = 'Plafond'
str5 = 'Oustanding'
str6 = 'Business_Type'
str7 = 'OpenDate'
str8 = 'LOB'
str9 = 'DelQ'
str10 = 'Collateral_Type'
str11 = 'Collateral_Value'

df2 = df.groupby([str2]).agg({ str3:'max'
                              ,str4:'max'
                              ,str5:'max'
                              ,str6:'max'
                              ,str7:'max'
                              ,str8:'max'
                              ,str9:'max'
                              ,str10:'max'
                              ,str11:'max'})[[str3
                                             ,str4
                                             ,str5
                                             ,str6
                                             ,str7
                                             ,str8
                                             ,str9
                                             ,str10
                                             ,str11]].reset_index()
print(df2)

                       CIF  Collectibility    Plafond  Oustanding  \
0   01.40.010999.03 / 7239               1   40000000    15360000   
1   01.40.011014.01 / 7281               1  100000000    70848600   
2   01.40.011029.01 / 7318               1  150000000   125000000   
3   01.40.011031.02 / 7328               1  250000000   187499800   
4   01.40.011039.03 / 7345               1  200000000   172220000   
5   01.40.011046.02 / 7365               1  300000000   250000000   
6   01.40.011047.05 / 7375               1  225000000   210937500   
7   01.40.011067.03 / 7424               1  100000000   100000000   
8   01.40.011071.02 / 7430               1  150000000   150000000   
9   01.40.011080.14 / 7456               1   20000000    14996000   
10  01.40.011088.08 / 7471               1  200000000   200000000   
11  01.40.011089.01 / 7472               1   75000000    68748500   
12  01.40.011099.01 / 7523               1  160000000   133165000   
13  01.40.011110.01 / 7541        

In [5]:
## Additional coloumn ##
df2["Sisa_Plafond"] = df2[str4] - df2[str5]

## Bucket Dpd - range mapping value ## 
bins = [0, 1, 30, 60, 90, 120, 150, 180]
names = ['1_Current', '2_DPD1+', '3_DPD30+', '4_DPD60+', 
         '5_DPD90+', '6_DPD120+', '7_DPD150+', '8_DPD180+']
d = dict(enumerate(names, 1))
df2['Bucket'] = np.vectorize(d.get)(np.digitize(df2[str9], bins))

In [6]:
## Kolektibitas mapping ## 
bins = [1, 2, 5]
names = ['1_Current', '2_SM', '3_NPL']
d = dict(enumerate(names, 1))
df2['Collectibility_Desc'] = np.vectorize(d.get)(np.digitize(df2[str3], bins))

In [7]:
## Oustanding ##
bins = [0, 50000000, 80000000, 100000000, 250000000, 350000000]
names = ['A_0-50 Jt', 'B_>50-80 Jt', 'C_>80-100 Jt', 
         'D_>100-250 Jt', 'E_>250-350 Jt', 'F_>350 Jt']
d = dict(enumerate(names, 1))
df2['Oustanding_Band'] = np.vectorize(d.get)(np.digitize(df2[str5], bins)) 

In [8]:
## Temporary ##
Collectibility = df2['Collectibility'].astype(int)
Collateral_Value = df2['Collateral_Value'].astype(float)
OS = df2["Oustanding"].astype(int)

In [9]:
# df2['Collateral_remains'] = OS - Collateral_Value
df2.loc[(Collateral_Value < OS), 'Collateral_remains'] = OS - Collateral_Value
df2.loc[(Collateral_Value > OS), 'Collateral_remains'] = 0
df2.loc[(Collateral_Value == OS), 'Collateral_remains'] = 0
df2.dropna(inplace=True)

In [10]:
## Grouping tipe bisnis ##
Business_Type = df2['Business_Type'].astype(int)
df2.loc[(Business_Type.isin([9990,1019,6000,1007])), 'Type_Buss'] = Business_Type
df2.loc[(Business_Type.isin([1001,1004,1008,1015,1020,8000,9000,1014,1016])), 'Type_Buss'] = 'Oth'
df2.loc[(Business_Type > 150000), 'Type_Buss'] = 'Oth'

In [11]:
## Create Debtor current month ##
DirResult = "C:\Data\BTN_DebiturInfo.csv"
df2.to_csv( DirResult, index = False, sep='|')
df_DebiturInfo = df2
# print(df_DebiturInfo.head(3))
print('Membuat dataset profile debitur selama 1 tahun - selesai')

Membuat dataset profile debitur selama 1 tahun - selesai


In [12]:
df_Dash1 = df_DebiturInfo
df_Dash1 = df_Dash1.groupby('Collectibility').agg({'Oustanding': lambda x: x.count()})
df_Dash1.reset_index(inplace=True)
df_Dash1 = df_Dash1.rename(columns={'Collectibility': 'Dash_Id','Oustanding': 'Total'})
print(df_Dash1)

   Dash_Id  Total
0        1     23


In [13]:
df_Dash2 = df_DebiturInfo
df_Dash2 = df_Dash2.groupby('Type_Buss').agg({'Oustanding': lambda x: x.count()})
df_Dash2.reset_index(inplace=True)
df_Dash2 = df_Dash2.rename(columns={'Type_Buss': 'Dash_Id','Oustanding': 'Total'})
print(df_Dash2)

  Dash_Id  Total
0    1007      8
1    1019      5
2    6000      4
3    9990      2
4     Oth      4


In [17]:
df_Dash3 = df_DebiturInfo
df_Dash3 = df_Dash3.groupby('Bucket').agg({'Oustanding': lambda x: x.count()})
df_Dash3.reset_index(inplace=True)
df_Dash3 = df_Dash3.rename(columns={'Bucket': 'Dash_Id','Oustanding': 'Total'})
print(df_Dash3)

     Dash_Id  Total
0  1_Current      3
1    2_DPD1+     14
2   3_DPD30+      6


In [14]:
df_Dash4 = df_DebiturInfo
df_Dash4 = df_Dash4.groupby('Collectibility_Desc').agg({'Oustanding': lambda x: x.count()})
df_Dash4.reset_index(inplace=True)
df_Dash4 = df_Dash4.rename(columns={'Collectibility_Desc': 'Dash_Id','Oustanding': 'Total'})
print(df_Dash4)

     Dash_Id  Total
0  1_Current     23


In [15]:
df_Dash5 = df_DebiturInfo
df_Dash5 = df_Dash5.groupby('Oustanding_Band').agg({'Oustanding': lambda x: x.count()})
df_Dash5.reset_index(inplace=True)
df_Dash5 = df_Dash5.rename(columns={'Oustanding_Band': 'Dash_Id','Oustanding': 'Total'})
print(df_Dash5)

         Dash_Id  Total
0      A_0-50 Jt      6
1    B_>50-80 Jt      3
2   C_>80-100 Jt      1
3  D_>100-250 Jt     11
4  E_>250-350 Jt      2


In [18]:
result = pd.concat([df_Dash1,df_Dash2,df_Dash3,df_Dash4,df_Dash5])
result = result.loc[:,~result.columns.duplicated()]
result.reset_index(drop=True, inplace=True)
result = result[['Dash_Id','Total']]
result = result.set_index('Dash_Id')
print(result)

               Total
Dash_Id             
1                 23
1007.0             8
1019.0             5
6000.0             4
9990.0             2
Oth                4
1_Current          3
2_DPD1+           14
3_DPD30+           6
1_Current         23
A_0-50 Jt          6
B_>50-80 Jt        3
C_>80-100 Jt       1
D_>100-250 Jt     11
E_>250-350 Jt      2
